# R_16 Create Data for final Data Analysis

In [12]:
import pandas as pd
import json

In [2]:
sudo_election_data = pd.read_csv('../data/curated/DataAnalysis/By_Electorate_SUDO_Political_Data.csv')

# Create API

In [3]:
import requests
import time

username = 'group9_admin'
password = 'group9_H1'

In [4]:
def make_get_request(username, password, url, params = {}):
    """ Helper to make the GET request using the 
            username - str
            password - str
            url - str (e.g. http://172.26.133.251:5984/tweet_database/_design/date/_view/date_electorate__nsentiment)
            params - dict (e.g. {'group': 'True'})"""
    
    start = time.time()

    # Set the authentication credentials
    auth = (username, password)

    # Send GET request using requests library
    response = requests.get(url, params=params, auth=auth)
    
    end = time.time()
    print(end-start)

    if response.ok:
        return response.json()
    else:
        return 'Error: '+ response.status_code

In [5]:
def GET_date_electorate__avgsent__pol(startdate, enddate, username, password):
    url = 'http://172.26.133.251:5984/tweet_database/_design/date/_view/date_electorate__avgsent__pol'

    # Set the parameters for the GET request
    params = {
        'group': 'True',
        'startkey': f'["{startdate}"]',
        'endkey': f'["{enddate}"]'
    }

    returned_data_json = make_get_request(username, password, url, params)
    
    return returned_data_json

In [6]:
date_electorate__avgsent__pol = GET_date_electorate__avgsent__pol('2022-02-09', '2022-05-22', username, password) # Pre Election

0.675678014755249


In [7]:
electorate_avg_sent_pol_sum_count = {}

for i in range(len(date_electorate__avgsent__pol['rows'])):
    
    electorate = date_electorate__avgsent__pol['rows'][i]['key'][1]
    if electorate not in electorate_avg_sent_pol_sum_count:
        electorate_avg_sent_pol_sum_count[electorate] = [0, 0]
    
    electorate_avg_sent_pol_sum_count[electorate][0] += date_electorate__avgsent__pol['rows'][i]['value']['sum']
    electorate_avg_sent_pol_sum_count[electorate][1] += date_electorate__avgsent__pol['rows'][i]['value']['count']

electorate_avg_sent = {key:electorate_avg_sent_pol_sum_count[key][0]/electorate_avg_sent_pol_sum_count[key][1] for key in electorate_avg_sent_pol_sum_count}
electorate_avg_sent

{'Ballarat': 0.09604587332053745,
 'Barker': 0.13687452229299357,
 'Barton': -0.031849999999999976,
 'Bass': 0.13327270471464023,
 'Bendigo': 0.2440960330578513,
 'Blaxland': -0.020776190476190476,
 'Bonner': 0.45026666666666665,
 'Boothby': 0.1808399267895879,
 'Bowman': 0.21097865168539326,
 'Braddon': 0.08518762376237624,
 'Burt': 0.09934469084268197,
 'Calare': 0.03591672200422578,
 'Canberra': 0.17303408594319009,
 'Capricornia': 0.12475576208178439,
 'Casey': 0.13820799999999997,
 'Corangamite': 0.1614812631578948,
 'Cowper': 0.12902387387387385,
 'Cunningham': 0.20348792588164977,
 'Dawson': 0.1018962555066079,
 'Deakin': 0.5243583333333334,
 'Dickson': 0.37930833333333336,
 'Dobell': 0.1776373522458629,
 'Durack': 0.11326254545454548,
 'Eden-Monaro': 0.11384502712477394,
 'Fadden': 0.11887238791971486,
 'Fairfax': 0.06783322841726616,
 'Farrer': 0.1434188741721854,
 'Fisher': 0.18124454545454544,
 'Flynn': 0.11101744548286605,
 'Forde': 0.26498374999999996,
 'Forrest': 0.200491

In [8]:
sentiment_df = pd.DataFrame({'electorate': electorate_avg_sent.keys(), 'sentiment':electorate_avg_sent.values()})

In [9]:
sudo_election_data_with_sentiment = sudo_election_data.merge(right = sentiment_df, how = 'inner', left_on='DivisionName', right_on='electorate')

In [10]:
sudo_election_data_with_sentiment.to_csv('../data/curated/DataAnalysis/By_Electorate_SUDO_Political_Sentiment_Data.csv', index = False)

In [13]:
electorates = list(electorate_avg_sent.keys())
electorates.sort()

with open('../data/curated/GeoJson/electorate_list.json', 'w') as f:
    json.dump(electorates, f)